In [8]:
!wget https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip

!wget https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip

--2023-06-13 15:47:59--  https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/brendenlake/omniglot/master/python/images_evaluation.zip [following]
--2023-06-13 15:47:59--  https://raw.githubusercontent.com/brendenlake/omniglot/master/python/images_evaluation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6462886 (6.2M) [application/zip]
Saving to: ‘images_evaluation.zip.3’

images_evaluation.z 100%[===================>]   6.16M  --.-KB/s    in 0.07s   

2023-06-13 15:47:59 (88.4 MB/s) - ‘images_evaluation.zip.3’ 

In [13]:
!unzip -qq images_background.zip
!unzip -qq images_evaluation.zip

In [ ]:
# -*- coding: utf-8 -*-
"""The correctblog.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1jyn0NmXxvyCweTJNX6NeLaaKkOSi6BtK
"""

# Commented out IPython magic to ensure Python compatibility.
# %pip install scipy
# %pip install scikit-learn

# Commented out IPython magic to ensure Python compatibility.
import sys
import numpy as np
# import pandas as pd
from matplotlib.pyplot import imread
import pickle
import os
import matplotlib.pyplot as plt
# %matplotlib inline

# import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from tensorflow.keras.layers import BatchNormalization, Concatenate
# from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
# from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from tensorflow.python.keras.layers import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

# !wget https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip
#
# !wget https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip
#
# !unhzip -qq images_background.zip
# !unzip -qq images_evaluation.zip

train_folder = './images_background'
val_folder = './images_evaluation'
save_path = './data'

t_total = time.time()
nmber_shots = 1

# Hyper parameters
evaluate_every = 50 # interval for evaluating on few shot tasks
batch_size = 32
n_iter = 800 # No. of training iterations
N_way = 5 # how many classes for testing few shot tasks
n_val = 250 # how many few shot tasks to validate on
best = -1

trials = 50
N=5

def loadimgs(path,n = 0):
    '''
    path => Path of train directory or test directory
    '''
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    # we load every alphabet seperately so we can isolate them later
    for alphabet in os.listdir(path):
        print("loading alphabet: " + alphabet)
        lang_dict[alphabet] = [curr_y,None]
        alphabet_path = os.path.join(path,alphabet)
        # every letter/category has it's own column in the array, so  load seperately
        for letter in os.listdir(alphabet_path):
            cat_dict[curr_y] = (alphabet, letter)
            category_images=[]
            letter_path = os.path.join(alphabet_path, letter)
            # read all the images in the current category
            for filename in os.listdir(letter_path):
                image_path = os.path.join(letter_path, filename)
                image = imread(image_path)
                category_images.append(image)
                y.append(curr_y)
            try:
                X.append(np.stack(category_images))
            # edge case  - last one
            except ValueError as e:
                print(e)
                print("error - category_images:", category_images)
            curr_y += 1
            lang_dict[alphabet][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,lang_dict

X,y,c=loadimgs(train_folder)

with open(os.path.join(save_path,"train.pickle"), "wb") as f:
    pickle.dump((X,c),f)

Xval,yval,cval=loadimgs(val_folder)

with open(os.path.join(save_path,"val.pickle"), "wb") as f:
    pickle.dump((Xval,cval),f)

def initialize_weights(shape, name=None, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

def initialize_bias(shape, name=None, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """

    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)

    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))

    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)

    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])

    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)

    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

    # return the model
    return siamese_net

model = get_siamese_model((105, 105, 1))
model.summary()

optimizer = Adam(learning_rate = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

with open(os.path.join(save_path, "train.pickle"), "rb") as f:
    (Xtrain, train_classes) = pickle.load(f)

print("Training alphabets: \n")
print(list(train_classes.keys()))

with open(os.path.join(save_path, "val.pickle"), "rb") as f:
    (Xval, val_classes) = pickle.load(f)

print("Validation alphabets:", end="\n\n")
print(list(val_classes.keys()))


def get_batch(batch_size, s="train"):
    """Create batch of n pairs, half same class, half different class"""
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape

    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes, size=(batch_size,), replace=False)

    # initialize 2 empty arrays for the input image batch
    pairs = [np.zeros((batch_size, h, w, 1)) for i in range(2)]

    # initialize vector for the targets
    targets = np.zeros((batch_size,))

    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size // 2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i, :, :, :] = X[category, idx_1].reshape(w, h, 1)
        idx_2 = rng.randint(0, n_examples)

        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category
        else:
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1, n_classes)) % n_classes

        pairs[1][i, :, :, :] = X[category_2, idx_2].reshape(w, h, 1)

    return pairs, targets

def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

def make_oneshot_task(N, k, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape

    indices = rng.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low,high),size=(N,),replace=False)

    else: # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,),replace=False)
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories, indices, :, :]
    targets = np.zeros((N,))
    counter = 0 # represents an index in the list
    for i in range(0,N):
        cate = categories[i]
        counter = 0
        for ik in range(0,k):
            s = rng.choice(n_examples, replace=False, size=(1,))
            support_set[i, ik :, :] = X[cate, s]
            targets[counter] = 1 # indicate true_catergory in tagert
            counter += 1 #next index
    # support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    # targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

def test_oneshot(model, N, k, k_shot, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way {} shot learning tasks ... \n".format(k,N, nmber_shots))
    for i in range(k):
        inputs, targets = make_oneshot_task(N, k_shot,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way {} shot learning accuracy \n".format(percent_correct,N, nmber_shots))
    return percent_correct


model_path = './weights'

print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss))
        val_acc = test_oneshot(model, N_way, n_val, nmber_shots, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

model.load_weights(os.path.join(model_path, "weights.{}.h5".format(n_iter)))

def nearest_neighbour_correct(pairs,targets):
    """returns 1 if nearest neighbour gets the correct answer for a one-shot task
        given by (pairs, targets)"""
    L2_distances = np.zeros_like(targets)
    for i in range(len(targets)):
        L2_distances[i] = np.sum(np.sqrt(pairs[0][i]**2 - pairs[1][i]**2))
    if np.argmin(L2_distances) == np.argmax(targets):
        return 1
    return 0

def test_nn_accuracy(N_ways,k, n_trials):
    """Returns accuracy of NN approach """
    print("Evaluating nearest neighbour on {} unique {} way {}-shot learning tasks ...".format(n_trials,N_ways,nmber_shots))

    n_right = 0

    for i in range(n_trials):
        pairs,targets = make_oneshot_task(N_ways, k,"val")
        correct = nearest_neighbour_correct(pairs,targets)
        n_right += correct
    return 100.0 * n_right / n_trials

ways = np.arange(19,20,2)
ways = 5
resume =  False


val_accs, train_accs,nn_accs = [], [], []
# for N in ways:

val_accs.append(test_oneshot(model, N, trials, nmber_shots, "val", verbose=True))
train_accs.append(test_oneshot(model, N, trials, nmber_shots, "train", verbose=True))
    # nn_acc = test_nn_accuracy(N, trials)
    # nn_accs.append(nn_acc)
    # print ("NN Accuracy = ", nn_acc)
print("---------------------------------------------------------------------------------------------------------------")

print( "total time ", (time.time()-t_total)/60.0)
with open(os.path.join(save_path,"accuracies.pickle"), "wb") as f:
    pickle.dump((val_accs,train_accs,nn_accs),f)

with open(os.path.join(save_path, "accuracies.pickle"), "rb") as f:
    (val_accs, train_accs, nn_accs) = pickle.load(f)
#
# def concat_images(X):
#     """Concatenates a bunch of images into a big matrix for plotting purposes."""
#     nc, h , w, _ = X.shape
#     X = X.reshape(nc, h, w)
#     n = np.ceil(np.sqrt(nc)).astype("int8")
#     img = np.zeros((n*w,n*h))
#     x = 0
#     y = 0
#     for example in range(nc):
#         img[x*w:(x+1)*w,y*h:(y+1)*h] = X[example]
#         y += 1
#         if y >= n:
#             y = 0
#             x += 1
#     return img
#
# def plot_oneshot_task(pairs):
#     fig,(ax1,ax2) = plt.subplots(nrows=1, ncols=2)
#     ax1.matshow(pairs[0][0].reshape(105,105), cmap='gray')
#     img = concat_images(pairs[1])
#     ax1.get_yaxis().set_visible(False)
#     ax1.get_xaxis().set_visible(False)
#     ax2.matshow(img,cmap='gray')
#     plt.xticks([])
#     plt.yticks([])
#     plt.show()
#
# # Example of concat image visualization
# pairs, targets = make_oneshot_task(16,"train","Sanskrit")
# plot_oneshot_task(pairs)
#
# fig,ax = plt.subplots(1)
# ax.plot(ways, val_accs, "m", label="Siamese(val set)")
# ax.plot(ways, train_accs, "y", label="Siamese(train set)")
# plt.plot(ways, nn_accs, label="Nearest neighbour")
#
# ax.plot(ways, 100.0/ways, "g", label="Random guessing")
# plt.xlabel("Number of possible classes in one-shot tasks")
# plt.ylabel("% Accuracy")
# plt.title("Omiglot One-Shot Learning Performance of a Siamese Network")
# box = ax.get_position()
# ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# inputs,targets = make_oneshot_task(20, "val", 'Oriya')
# plt.show()
#
# plot_oneshot_task(inputs)
#
#
#
#
#
#
#


loading alphabet: Braille
loading alphabet: Gujarati
loading alphabet: Grantha
loading alphabet: Tifinagh
loading alphabet: Armenian
loading alphabet: Burmese_(Myanmar)
loading alphabet: Asomtavruli_(Georgian)
loading alphabet: Balinese
loading alphabet: Hebrew
loading alphabet: Inuktitut_(Canadian_Aboriginal_Syllabics)
loading alphabet: Mkhedruli_(Georgian)
loading alphabet: Tagalog
loading alphabet: Japanese_(katakana)
loading alphabet: Latin
loading alphabet: Ojibwe_(Canadian_Aboriginal_Syllabics)
loading alphabet: Blackfoot_(Canadian_Aboriginal_Syllabics)
loading alphabet: Korean
loading alphabet: Anglo-Saxon_Futhorc
loading alphabet: Greek
loading alphabet: Malay_(Jawi_-_Arabic)
loading alphabet: Alphabet_of_the_Magi
loading alphabet: Sanskrit
loading alphabet: Arcadian
loading alphabet: Futurama
loading alphabet: Bengali
loading alphabet: Early_Aramaic
loading alphabet: Cyrillic
loading alphabet: Japanese_(hiragana)
loading alphabet: N_Ko
loading alphabet: Syriac_(Estrangelo)
loa